In [1]:
import numpy as np
import pulp
import math
import copy
from scipy.stats import weibull_min
from scipy.special import gamma


In [2]:
LOWER_BOUND_transport = 8
UPPER_BOUND_transport = 15

LOWER_BOUND_demand = 100
UPPER_BOUND_demand = 400

LOWER_BOUND_production = 15
UPPER_BOUND_production = 25

LOWER_BOUND_service = 15
UPPER_BOUND_service = 25

LOWER_BOUND_extra_charge = 40
UPPER_BOUND_extra_charge = 60

LOWER_BOUND_safety_stock = 0
UPPER_BOUND_safety_stock = 600

LOWER_BOUND_service_add = 20
UPPER_BOUND_service_add = 30

LOWER_BOUND_service_stocks = 1
UPPER_BOUND_service_stocks = 3

LOWER_BOUND_production_stop = 30
UPPER_BOUND_production_stop = 50



In [3]:
def generate_data(n, m, verbose=True):

    d_k = np.random.randint(LOWER_BOUND_demand, UPPER_BOUND_demand,  size = n) #спрос в k узле
    D = np.sum(d_k) # весь спрос

#генерация вместимостей для РЦ 
    mu = np.zeros(2, dtype=int)
    mu[0] = np.random.randint(int(0.3*D), int(0.7*D))
    mu[1] = D - mu[0]
    #генерация возможных расширений пропускных способностей
    mu_add = np.zeros(2, dtype=int)
    for j in range(2):
        mu_add[j] = np.random.choice([0, int(0.1*mu[j]), int(0.3*mu[j]), int(0.5*mu[j]), int(0.7*mu[j])])
        

#генерация мощностей производственных центров
    nu = np.zeros(m, dtype=int)
    remaining = D
    #распределение между ПЦ
    for i in range(m - 1):
        min_val = max(1, int(0.3 * remaining))
        max_val = min(int(0.7 * remaining), remaining - (m - i - 1))
        nu[i] = np.random.randint(min_val, max_val + 1)
        remaining -= nu[i]
    nu[-1] = remaining  #последний ПЦ получает остаток
    #генерация возможных повышений мощностей
    nu_add = np.array([
        np.random.choice([0, int(0.1 * nu_i), int(0.3*nu_i), int(0.5*nu_i), int(0.7*nu_i), int(nu_i), int(1.2*nu_i)])
        for nu_i in nu
    ])

    Cp_i = np.random.randint(LOWER_BOUND_production, UPPER_BOUND_production,  size = m) #затраты на производство единицы продукции для i производственного центра
    Cd_j = np.random.randint(LOWER_BOUND_service, UPPER_BOUND_service, size = 2) #затраты на единицу продукции для j распределительного центра
    Cd_j_add = np.zeros(2, dtype=int)
    for j in range(2):
        Cd_j_add[j] = np.random.choice([int(1.1 * Cd_j[j]), int(1.3*Cd_j[j]), int(1.5*Cd_j[j])])
    Cp_i_add = np.zeros(m, dtype=int)
    for i in range(m):
        Cp_i_add[i] = np.random.choice([Cp_i[i], Cp_i[i] + np.random.choice([int(0.1 * Cp_i[i]), int(0.3*Cp_i[i]), int(0.5*Cp_i[i])])])

    Cs_k = np.random.randint(LOWER_BOUND_service, UPPER_BOUND_service, size = n) #затраты на единицу продукции для k узла спроса
    Ct_ij = np.random.randint(LOWER_BOUND_transport, UPPER_BOUND_transport, size = (m, 2)) #транспортные затраты на единицу продукции от i производственного центра к j распределительному центру
    Cr_jk = np.random.randint(LOWER_BOUND_transport, UPPER_BOUND_transport, size = (2, n)) #транспортные затраты на единицу продукции от j распределительного центра к k узлу спроса
    Cr_jk_transposed = np.transpose(Cr_jk)
    C_stop_i = np.random.randint(LOWER_BOUND_production_stop, UPPER_BOUND_production_stop, size = m) # затраты на остановку производства i за единицу продукции
    C_over_do_k = np.random.randint(LOWER_BOUND_service, UPPER_BOUND_service) # затраты на единицу лишней продукции
    extra_charge_percent = np.random.randint(LOWER_BOUND_extra_charge, UPPER_BOUND_extra_charge, size = n)
    extra_charge = extra_charge_percent/100 # наценка узла спроса

    alpha_prop = np.random.randint(LOWER_BOUND_safety_stock, UPPER_BOUND_safety_stock, size = m)/100  # вместимость страхового запаса в производственном узле i
    beta_prop = np.random.randint(LOWER_BOUND_safety_stock, UPPER_BOUND_safety_stock, size = 2)/100 # вместимость страхового запаса в распределительном узле j
    gamma_prop = np.random.randint(LOWER_BOUND_safety_stock, UPPER_BOUND_safety_stock, size = n)/100 # вместимость страхового запаса в узле спроса k
    
    alpha = np.floor(alpha_prop * nu)
    beta = np.floor(beta_prop * mu)
    gamma = np.floor(gamma_prop * d_k)

    Hp_i = np.random.randint(LOWER_BOUND_service_stocks, UPPER_BOUND_service_stocks, size = m) # стоимость хранения единицы продукции в производственном узле i в единицу времени
    Hd_j = np.random.randint(LOWER_BOUND_service_stocks, UPPER_BOUND_service_stocks, size = 2)# стоимость хранения единицы продукции в распределительном узле j в единицу времени
    Hs_k = np.random.randint(LOWER_BOUND_service_stocks, UPPER_BOUND_service_stocks, size = n)# стоимость хранения единицы продукции в узле спроса k в единицу времени
    
    # alpha = np.zeros(m, dtype = int)
    # beta = np.zeros(2, dtype = int)
    # gamma = np.zeros(n, dtype = int)


    data = {
        "d_k": d_k,
        "D": D,
        "mu": mu,
        "mu_add": mu_add,
        "nu": nu,
        "nu_add": nu_add,
        "Cp_i": Cp_i,
        "Cd_j": Cd_j,
        "Cd_j_add": Cd_j_add,
        "Cp_i_add": Cp_i_add,
        "Cs_k": Cs_k,
        "Ct_ij": Ct_ij,
        "Cr_jk": Cr_jk,
        "Cr_jk_transposed": Cr_jk_transposed,
        'C_stop_i': C_stop_i,
        'extra_charge': extra_charge,
        'alpha': alpha,
        'beta': beta,
        'gamma': gamma,
        'C_over_do_k': C_over_do_k,
        'Hp_i': Hp_i,
        'Hd_j': Hd_j,
        'Hs_k': Hs_k
    }

    #if verbose:
        #print("Сгенерированные данные:")
        #for key, value in data.items():
            #print(f"{key}: {value}")
    
    return data

In [4]:
def convert_numpy_to_python(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)
    elif isinstance(obj, dict):
        return {key: convert_numpy_to_python(value) for key, value in obj.items()}
    else:
        return obj
    
def flatten_data(data):
    flat_data = {}
    for key, value in data.items():
        if isinstance(value, list):
            # Если элемент - матрица
            if all(isinstance(x, list) for x in value):
                for i, row in enumerate(value):
                    for j, val in enumerate(row):
                        flat_data[f"{key}_{i}_{j}"] = val
            # Если просто список
            else:
                for i, val in enumerate(value):
                    flat_data[f"{key}_{i}"] = val
        # Если одиночное значение
        else:
            flat_data[key] = value
    return flat_data

In [5]:
#целочисленное линейное программирование 
def optimal_transportation(node_capacity, distr_capacity, cost_of_roads):
    try:
        # задача сбалансирована по условию (спрос = предлодению), проверка не нужна
        # if sum(node_capacity) != sum(distr_capacity):
        #     raise ValueError

        num_nodes = len(node_capacity)
        num_distr = len(distr_capacity)

        # задача минимизации
        prob = pulp.LpProblem("Transportation_Problem", pulp.LpMinimize)

        # матрица неотрицательный целочисленных переменных
        x = [[pulp.LpVariable(f"x_{i}_{j}", lowBound=0, cat='Integer') 
              for j in range(num_distr)] 
              for i in range(num_nodes)]

        # целевая функция: сумма стоимостей * x_ij (ее хотим минимизировать)
        prob += pulp.lpSum(cost_of_roads[i][j] * x[i][j] 
                          for i in range(num_nodes) 
                          for j in range(num_distr))

        # ограничения:
        # 1. сумма перевозок от узла i = его мощности
        for i in range(num_nodes):
            prob += pulp.lpSum(x[i][j] for j in range(num_distr)) == node_capacity[i]

        # 2. сумма перевозок в РЦ j = его вместимости
        for j in range(num_distr):
            prob += pulp.lpSum(x[i][j] for i in range(num_nodes)) == distr_capacity[j]

        prob.solve(pulp.PULP_CBC_CMD(msg=False))

        if pulp.LpStatus[prob.status] != 'Optimal':
            print("Оптимальное целочисленное решение не найдено.")
            return None, None

        # матрица перевозок и общая стоимость
        transport_matrix = [[int(pulp.value(x[i][j])) for j in range(num_distr)] 
                           for i in range(num_nodes)]
        total_cost = int(pulp.value(prob.objective))

        return total_cost, transport_matrix

    except Exception as e:
        print(f"Ошибка: {e}")
        return None, None

In [ ]:
n = 4  # число узлов спроса
m = 3  # число производственных центров
generated_data =  generate_data(n, m)
#распределяем продукцию из пц по рц
transp_cost_between_prod_and_distr, transp_matrix_between_prod_and_distr = optimal_transportation( generated_data["nu"], generated_data["mu"], generated_data["Ct_ij"])
transp_cost_between_distr_and_demand, transp_matrix_between_distr_and_demand = optimal_transportation( generated_data["d_k"], generated_data["mu"], generated_data["Cr_jk_transposed"])
all_gen_data = convert_numpy_to_python(generated_data)


E_d = np.zeros(2) # стоимость единицы товара в j распределительном центре
E_s = np.zeros(n) # стоимость единицы товара в k узле спроса
sum_1 = np.zeros(m)
sum_2 = np.zeros(2)
dot_2 = np.zeros(n)
for j in range(2):
    for i in range(m):
        sum_1[i] = (all_gen_data['Cp_i'][i] + all_gen_data['Ct_ij'][i][j] + all_gen_data['Cd_j'][j])* transp_matrix_between_prod_and_distr[i][j]
    E_d[j] = np.round((np.sum(sum_1))/all_gen_data['mu'][j], decimals = 2)
for k in range(n):
    for j in range(2):
        sum_2[j] = ((E_d[j] + all_gen_data['Cr_jk'][j][k] + all_gen_data['Cs_k'][k]))
    dot_2 = sum_2 * transp_matrix_between_distr_and_demand[k]
    E_s[k] = np.round(np.sum(dot_2)/all_gen_data['d_k'][k], decimals = 2)
s_k = np.round(E_s * (1 + np.array(all_gen_data['extra_charge'])), decimals = 2)


profit_per_unit = np.round(s_k - E_s, decimals = 2)
cost_1 = sum(np.array(all_gen_data['nu']) * np.array(all_gen_data['Cp_i']))
cost_2 = sum(np.array(all_gen_data['Cd_j']) * np.array(all_gen_data['mu']))
cost_3 = sum(np.array(all_gen_data['Cs_k']) * np.array(all_gen_data['d_k']))
cost_4 = sum(sum(np.array(all_gen_data['Ct_ij']) * transp_matrix_between_prod_and_distr))
cost_5 = sum(sum(np.array(all_gen_data['Cr_jk_transposed']) * transp_matrix_between_distr_and_demand))
total_cost = cost_1 + cost_2 + cost_3 + cost_4 + cost_5
# print('total_cost', total_cost)
total_value = np.round(sum(s_k * np.array(all_gen_data['d_k'])), decimals = 2)
# print('total_value', total_value)
total_profit = np.round(total_value - total_cost, decimals = 2)
# print('total_profit', total_profit) 


all_gen_data = {
    "total_cost": total_cost,
    "total_value": total_value,
    "total_profit": total_profit,
    "d_k": all_gen_data["d_k"],
    "D": all_gen_data["D"],
    "mu": all_gen_data["mu"],
    "mu_add": all_gen_data["mu_add"],
    "nu": all_gen_data["nu"],
    "nu_add": all_gen_data["nu_add"],
    "Cp_i": all_gen_data["Cp_i"],
    "Cd_j": all_gen_data["Cd_j"],
    "Cp_i_add": all_gen_data['Cp_i_add'],
    "Cd_j_add": all_gen_data["Cd_j_add"],
    "Cs_k": all_gen_data["Cs_k"],
    "Ct_ij": all_gen_data["Ct_ij"],
    "Cr_jk": all_gen_data["Cr_jk"],
    "Cr_jk_transposed": all_gen_data["Cr_jk_transposed"],
    'C_stop_i': all_gen_data["C_stop_i"],
    'extra_charge': all_gen_data['extra_charge'],
    'q_ij': transp_matrix_between_prod_and_distr,
    "d_star_jk": transp_matrix_between_distr_and_demand,
    "s_k": s_k.tolist(),
    "profit_per_unit": profit_per_unit,
    "E_d": E_d,
    "E_s": E_s, 
    'Hp_i': all_gen_data['Hp_i'],
    'Hd_j': all_gen_data['Hd_j'],
    'Hs_k': all_gen_data['Hs_k'],
    'alpha': all_gen_data["alpha"],
    'beta': all_gen_data["beta"],
    'gamma': all_gen_data["gamma"]
}
for key, value in all_gen_data.items():
    print(f"{key}: {value}")






In [7]:
# m, n = 3, 4
# all_gen_data = { 
#     'd_k':[120, 166, 250, 367], 
#     'D': 903,
#     'mu': [525, 378],
#     'mu_add': [262, 113],
#     'nu':[427, 233, 243],
#     'nu_add':   [18, 233, 0],

#     'Cp_i': [15, 19, 17],
#     'Cd_j': [24, 20],
#     'Cp_i_add': [16, 56, 30],
#     'Cd_j_add': [26, 22],
#     'Cs_k': [23, 15, 22, 15],
#     'Ct_ij':  [[14, 9], [12, 14], [15, 9]],
#     'Cr_jk':   [[14, 12, 8, 9], [9, 11, 13, 10]],
#     'Cr_jk_transposed': [[14, 9], [12, 11], [8, 13], [9, 10]],
#     'C_stop_i':  [46, 36, 42],
#     'extra_charge': [0.53, 0.42, 0.49, 0.43],

#     'Hp_i': [1, 2, 1],
#     'Hd_j': [2, 1],
#     'alpha': [866.0, 468.0, 702.0],
#     'beta': [399.0, 1901.0]}

# transp_cost_between_prod_and_distr, transp_matrix_between_prod_and_distr = optimal_transportation( all_gen_data["nu"], all_gen_data["mu"], all_gen_data["Ct_ij"])
# transp_cost_between_distr_and_demand, transp_matrix_between_distr_and_demand = optimal_transportation( all_gen_data["d_k"], all_gen_data["mu"], all_gen_data["Cr_jk_transposed"])

# # print(transp_matrix_between_prod_and_distr, transp_matrix_between_distr_and_demand)
# all_gen_data['q_ij'] = transp_matrix_between_prod_and_distr
# all_gen_data['d_star_jk'] = transp_matrix_between_distr_and_demand
# E_d = np.zeros(2)
# E_s = np.zeros(n) # стоимость единицы товара в k узле спроса
# sum_1 = np.zeros(m)
# sum_2 = np.zeros(2)
# dot_2 = np.zeros(n)
# for j in range(2):
#     for i in range(m):
#         sum_1[i] = (all_gen_data['Cp_i'][i] + all_gen_data['Ct_ij'][i][j] + all_gen_data['Cd_j'][j])* all_gen_data['q_ij'][i][j]
#     E_d[j] = np.round((np.sum(sum_1))/all_gen_data['mu'][j], decimals = 2)
# for k in range(n):
#     for j in range(2):
#         sum_2[j] = ((E_d[j] + all_gen_data['Cr_jk'][j][k] + all_gen_data['Cs_k'][k]))
#     dot_2 = sum_2 * all_gen_data['d_star_jk'][k]
#     E_s[k] = np.round(np.sum(dot_2)/all_gen_data['d_k'][k], decimals = 2)

# s_k = np.round(E_s * (1 + np.array(all_gen_data['extra_charge'])), decimals = 2)

# profit_per_unit = np.round(s_k - E_s, decimals = 2)
# cost_1 = sum(np.array(all_gen_data['nu']) * np.array(all_gen_data['Cp_i']))
# cost_2 = sum(np.array(all_gen_data['Cd_j']) * np.array(all_gen_data['mu']))
# cost_3 = sum(np.array(all_gen_data['Cs_k']) * np.array(all_gen_data['d_k']))
# cost_4 = sum(sum(np.array(all_gen_data['Ct_ij']) * all_gen_data['q_ij']))
# cost_5 = sum(sum(np.array(all_gen_data['Cr_jk_transposed']) * all_gen_data['d_star_jk']))
# total_cost = cost_1 + cost_2 + cost_3 + cost_4 + cost_5

# total_value = np.round(sum(s_k * np.array(all_gen_data['d_k'])), decimals = 2)

# total_profit = np.round(total_value - total_cost, decimals = 2)

# all_gen_data['E_s'] = E_s.tolist() 
# all_gen_data['E_d'] = E_d.tolist()  
# all_gen_data['s_k'] = s_k.tolist()
# all_gen_data['profit_per_unit'] = profit_per_unit.tolist()

# all_gen_data['total_cost'] = total_cost.tolist()
# all_gen_data['total_value'] = total_value.tolist()
# all_gen_data['total_profit'] = total_profit.tolist()

# for key, value in all_gen_data.items():
#     print(f"{key}: {value}")







In [8]:
# способы распределения продукции по узлам, когда предложение не удовлетворяет спросу, при сбое в РЦ 
# сценарий 1: получают те, которые привязаны к рабочему РЦ
def scenario_1(transp_matrix_without_broken_1, transp_matrix_without_broken_2, 
               transp_matrix_with_broken_1, data, Ct_ij, Cr_jk, index_of_working_distr_node, n):
    total_cost = np.round(
        sum(transp_matrix_without_broken_1 * np.array(data['Cp_i'])) + 
        (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
        sum(np.array(data['Cs_k']) * transp_matrix_without_broken_2) + 
        sum(x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
        sum(x * y for x, y in zip(Cr_jk, transp_matrix_without_broken_2)) + 
        sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals=2)
    
    total_value = np.round(sum(np.array(data['s_k']) * transp_matrix_without_broken_2), decimals=2)
    total_profit = np.round(total_value - total_cost, decimals=2)
    return total_profit, transp_matrix_without_broken_2

# сценарий 2: каждому узлу спроса одинаковое количество продукции
def scenario_2(transp_matrix_without_broken_1, transp_matrix_with_broken_1, 
              data, Ct_ij, Cr_jk, index_of_working_distr_node, n):
    h = np.zeros(n)
    for k in range(n):
        h[k] = data['s_k'][k] - (data['Cs_k'][k] + Cr_jk[k])
    sorted_indices = sorted(range(n), key=lambda i: (-h[i]))
    a = int(sum(transp_matrix_without_broken_1) / n)
    
    if all(x > a for x in data['d_k']):
        remainder = sum(transp_matrix_without_broken_1) - a * n
        add_q = np.zeros(n)
        
        for k in sorted_indices:
            if data['d_k'][k] - a >= 0 and remainder != 0:
                if data['d_k'][k] - a > remainder:
                    add_q[k] = remainder
                    remainder = 0
                else:
                    add_q[k] = data['d_k'][k] - a
                    remainder -= (data['d_k'][k] - a)
        
        a = add_q + a
        total_cost = np.round(
            sum(x * y for x, y in zip(transp_matrix_without_broken_1, np.array(data['Cp_i']))) + 
            (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
            sum(np.array(data['Cs_k']) * a) + 
            sum(x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
            sum(x * y for x, y in zip(a, Cr_jk)) + 
            sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals=2)
        
        total_value = np.round(sum(a * (np.array(data['s_k']))), decimals=2)
        total_profit = np.round(total_value - total_cost, decimals=2)
        return total_profit, a
    else:
        return None, None
    
#сценарий 3: пропорционально потребностям узлов спроса, используя весовые коэффициенты
def scenario_3(transp_matrix_without_broken_1, transp_matrix_with_broken_1, 
              data, Ct_ij, Cr_jk, index_of_working_distr_node, n):
    h = np.zeros(n)
    for k in range(n):
        h[k] = data['s_k'][k] - (data['Cs_k'][k] + Cr_jk[k])
    
    sorted_indices = sorted(range(n), key=lambda k: -h[k])
    W = np.array(data['d_k']) / data['D']
    prop = W * sum(transp_matrix_without_broken_1)
    int_prop = [math.floor(p) for p in prop]
    remainder = sum(transp_matrix_without_broken_1) - sum(int_prop)
    add_q = np.zeros(n)
    
    for k in sorted_indices:
        if data['d_k'][k] - int_prop[k] > 0 and remainder != 0:
            if data['d_k'][k] - int_prop[k] > remainder:
                add_q[k] = remainder
                remainder = 0
            else:
                add_q[k] = data['d_k'][k] - int_prop[k]
                remainder -= (data['d_k'][k] - int_prop[k])
    
    int_prop = add_q + int_prop
    total_cost = np.round(
        sum(x * y for x, y in zip(transp_matrix_without_broken_1, np.array(data['Cp_i']))) + 
        (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
        sum(np.array(data['Cs_k']) * int_prop) + 
        sum(x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
        sum(x * y for x, y in zip(int_prop, Cr_jk)) + 
        sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals=2)
    
    total_value = np.round(sum(int_prop * (np.array(data['s_k']))), decimals=2)
    total_profit = np.round(total_value - total_cost, decimals=2)
    return total_profit, int_prop

# сценарий 4: cортировка узлов спроса по наибольшей выручке для максимизации прибыли
def scenario_4(transp_matrix_without_broken_1, transp_matrix_with_broken_1, 
              data, Ct_ij, Cr_jk, index_of_working_distr_node, n):
    h = np.zeros(n)
    filling = np.zeros(n)
    remainder = sum(transp_matrix_without_broken_1)
    
    for k in range(n):
        h[k] = data['s_k'][k] - (data['Cs_k'][k] + Cr_jk[k])
    
    sorted_indices = sorted(range(n), key=lambda k: -h[k])
    
    for k in sorted_indices:
        if data['d_k'][k] > remainder and remainder != 0:
            filling[k] = remainder
            remainder = 0
        elif data['d_k'][k] < remainder and remainder != 0:
            filling[k] = data['d_k'][k]
            remainder -= data['d_k'][k]
    
    total_cost = np.round(
        sum(transp_matrix_without_broken_1 * np.array(data['Cp_i'])) + 
        (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
        sum(np.array(data['Cs_k']) * filling) + 
        sum(x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
        sum(x * y for x, y in zip(filling, Cr_jk)) + 
        sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals=2)
    
    total_value = np.round(sum(filling * (np.array(data['s_k']))), decimals=2)
    total_profit = np.round(total_value - total_cost, decimals=2)
    return total_profit, filling




In [9]:
# сбой в распределительном узле
def distribution_node_disruption(m, n, index_of_broken_distr_node, all_gen_data):
    index_of_working_distr_node = 1 - index_of_broken_distr_node
    transp_matrix_without_broken_1 = [row[index_of_working_distr_node] for row in all_gen_data['q_ij']]
    transp_matrix_with_broken_1 = [row[index_of_broken_distr_node] for row in all_gen_data['q_ij']]
    transp_matrix_without_broken_2 = [row[index_of_working_distr_node] for row in all_gen_data['d_star_jk']]
    Ct_ij = [row[index_of_working_distr_node] for row in all_gen_data['Ct_ij']]
    Cr_jk = [row[index_of_working_distr_node] for row in all_gen_data['Cr_jk_transposed']]
    #1. использование количества продукции, которое обычно поступает в рабочий рц
    print("1. использование количества продукции, которое обычно поступает в рабочий рц")
    profit1, dist1 = scenario_1(transp_matrix_without_broken_1, transp_matrix_without_broken_2,
                              transp_matrix_with_broken_1, all_gen_data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
    print('сценарий 1:', profit1)
    
    profit2, dist2 = scenario_2(transp_matrix_without_broken_1, transp_matrix_with_broken_1,
                              all_gen_data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
    if profit2 is not None:
        print('сценарий 2:', profit2)
    else:
        print('не можем использовать сценарий 2:')
    
    profit3, dist3 = scenario_3(transp_matrix_without_broken_1, transp_matrix_with_broken_1,
                              all_gen_data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
    print('сценарий 3:', profit3)
    
    profit4, dist4 = scenario_4(transp_matrix_without_broken_1, transp_matrix_with_broken_1,
                              all_gen_data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
    print('сценарий 4:', profit4)
    
    
    # 2. использование перенаправления 
    print("\n2. использование перенаправления ")
    # сценарий 6: перенаправление всей продукции в рабочий рц
    if ((np.array(all_gen_data['mu']) + np.array(all_gen_data['mu_add'])))[index_of_working_distr_node] >= all_gen_data['D']:
        total_cost_skript_6 = np.round(sum(np.array(all_gen_data['nu']) * np.array(all_gen_data['Cp_i'])) + 
                                       (np.array(all_gen_data['Cd_j'][index_of_working_distr_node]) * all_gen_data['mu'][index_of_working_distr_node]) + 
                                       (np.array(all_gen_data['Cd_j_add'][1 - index_of_working_distr_node]) * all_gen_data['mu'][1 - index_of_working_distr_node]) +
                                       sum(np.array(all_gen_data['Cs_k']) * np.array(all_gen_data['d_k']) ) + sum( x * y for x, y in zip(Ct_ij, all_gen_data['nu'])) + 
                                       sum( x * y for x, y in zip(Cr_jk, all_gen_data['d_k'])), decimals = 2)
        total_value_skript_6 = np.round(sum(np.array(all_gen_data['s_k']) * np.array(all_gen_data['d_k'])), decimals = 2)
        total_profit_skript_6 = np.round(total_value_skript_6 - total_cost_skript_6, decimals = 2)
        print('сценарий 6: перенаправляем все в рабочий рц:', total_profit_skript_6)  
    else:
        print("не можем использовать сценарий 6: перенаправление всей продукции в рабочий рц")
        total_profit_skript_6 = None


    # сценарий 7: перенаправление продукции с некоторых производственных узлов 
    if ((np.array(all_gen_data['mu']) + np.array(all_gen_data['mu_add'])))[index_of_working_distr_node] < all_gen_data['D']:
        u = np.zeros(m)
        add_prod = np.zeros(m)
        for i in range(m):
            u[i] = np.array(all_gen_data['Cp_i'])[i] + (np.array(Ct_ij))[i] - np.array(all_gen_data['C_stop_i'])[i]
        
        sorted_prod_indices = sorted(range(m), key=lambda i: -u[i])
        remainder_add_mu = all_gen_data['mu_add'][index_of_working_distr_node]
        
        for i in sorted_prod_indices:
            if (all_gen_data['nu'][i] - transp_matrix_without_broken_1[i]) < remainder_add_mu and remainder_add_mu != 0:
                add_prod[i] = all_gen_data['nu'][i]
                remainder_add_mu -= all_gen_data['nu'][i]
            elif (all_gen_data['nu'][i] - transp_matrix_without_broken_1[i]) >= remainder_add_mu and remainder_add_mu != 0:
                add_prod[i] = remainder_add_mu
                remainder_add_mu = 0
        new_transp_matrix = transp_matrix_without_broken_1 + add_prod
    

        print("сценарий 7: перенаправление продукции с некоторых производственных узлов: ")
        profit1_7, _ = scenario_1(new_transp_matrix, transp_matrix_without_broken_2, transp_matrix_with_broken_1 - add_prod, all_gen_data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
        profit1_7 = profit1_7 - np.sum(add_prod * np.array(all_gen_data['Cd_j_add'][1 - index_of_working_distr_node]))
        print('сценарий 7.1:', profit1_7)

        profit2_7, _ = scenario_2(new_transp_matrix, transp_matrix_with_broken_1 - add_prod, all_gen_data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
        if profit2_7 is not None:
            profit2_7 = np.round( profit2_7 - np.sum(add_prod * np.array(all_gen_data['Cd_j_add'][1 - index_of_working_distr_node])), decimals = 2)
            print('сценарий 7.2:', profit2_7)
        else: 
            print('не можем использовать сценарий 7.2')
            profit2_7 = None
        profit3_7, _ = scenario_3(new_transp_matrix, transp_matrix_with_broken_1 - add_prod, all_gen_data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
        profit3_7 =  np.round(profit3_7- np.sum(add_prod * np.array(all_gen_data['Cd_j_add'][1 - index_of_working_distr_node])), decimals = 2)
        print('сценарий 7.3:', profit3_7)
        profit4_7, _ = scenario_4(new_transp_matrix, transp_matrix_with_broken_1 - add_prod, all_gen_data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
        profit4_7 = np.round( profit4_7- np.sum(add_prod * np.array(all_gen_data['Cd_j_add'][1 - index_of_working_distr_node])), decimals = 2)
        print('сценарий 7.4:', profit4_7)
    else:
        profit1_7 = profit2_7 = profit3_7 = profit4_7 = 0
        
    rez = [profit1, profit2, profit3, profit4, profit1_7, profit2_7, profit3_7, profit4_7, total_profit_skript_6]
    return rez
    
        
     

In [ ]:
def scenario_1_new(number_of_available, number_of_demand_nodes, all_gen_data):
    a = int(np.sum(number_of_available) / number_of_demand_nodes)
    if all(x > a for x in all_gen_data['d_k']):
        remainder = np.sum(number_of_available) - a * number_of_demand_nodes
        add_q = np.zeros(number_of_demand_nodes)
        h = np.zeros(number_of_demand_nodes)
        for k in range(number_of_demand_nodes):
            h[k] = all_gen_data['s_k'][k] - (all_gen_data['Cs_k'] + all_gen_data['Cr_jk'])[k]
        sorted_indices = sorted(range(number_of_demand_nodes), key=lambda i: (-h[i]))
        for k in sorted_indices:
            if all_gen_data['d_k'][k] - a >= 0 and remainder != 0:
                if all_gen_data['d_k'][k] - a > remainder:
                    add_q[k] = remainder
                    remainder = 0
                else:
                    add_q[k] = all_gen_data['d_k'][k] - a
                    remainder -= (all_gen_data['d_k'][k] - a)
        a = add_q + a
        return a
    else:
        return None
    

def scenario_2_new(number_of_available, number_of_demand_nodes, all_gen_data):
    W = np.array(all_gen_data['d_k']) / all_gen_data['D']    
    prop = W * np.sum(number_of_available)
    int_prop = [math.floor(p) for p in prop]
    remainder_prop = np.sum(number_of_available) - sum(int_prop)
    add_q = np.zeros(number_of_demand_nodes)
    h = np.zeros(number_of_demand_nodes)
    for k in range(number_of_demand_nodes):
        h[k] = all_gen_data['s_k'][k] - (all_gen_data['Cs_k'] + all_gen_data['Cr_jk'])[k]
    sorted_indices = sorted(range(number_of_demand_nodes), key=lambda i: (-h[i]))
    for k in sorted_indices:
        if all_gen_data['d_k'][k] - int_prop[k] > 0 and remainder_prop != 0:
            if all_gen_data['d_k'][k] - int_prop[k] > remainder_prop:
                add_q[k] = remainder_prop
                remainder_prop = 0
            else:
                add_q[k] = all_gen_data['d_k'][k] - int_prop[k]
                remainder_prop -= (all_gen_data['d_k'][k] - int_prop[k])
    int_prop = add_q + int_prop
    return int_prop
    
def scenario_3_new(number_of_available, number_of_demand_nodes, all_gen_data):
    h = np.zeros(number_of_demand_nodes)
    filling = np.zeros(number_of_demand_nodes)
    remainder_h = np.sum(number_of_available)
    for k in range(number_of_demand_nodes):
        h[k] = all_gen_data['s_k'][k] - (all_gen_data['Cs_k'] + all_gen_data['Cr_jk'])[k]
    sorted_indices = sorted(range(number_of_demand_nodes), key=lambda k: -h[k])
    for k in sorted_indices:
        if all_gen_data['d_k'][k] > remainder_h and remainder_h != 0:
            filling[k] = remainder_h
            remainder_h = 0
        elif all_gen_data['d_k'][k] < remainder_h and remainder_h != 0:
            filling[k] = all_gen_data['d_k'][k]
            remainder_h -= all_gen_data['d_k'][k]
    return filling


In [11]:
def production_node_disruption(m, n, index_of_broken_prod_node, data):
    
    # 1. используем количество продукции, которое производит (m - 1) производственный узел
    #print("1. используем количество продукции, которое производит (m - 1) производственный узел")
    q_without_broken_prod_node_all = copy.deepcopy(data['q_ij'])
    q_without_broken_prod_node_all[index_of_broken_prod_node] = [0, 0]
    q_without_broken_prod_node = [sum(x) for x in zip(*q_without_broken_prod_node_all)]
    transp_matrix_stop = np.zeros(m) # так как не замедляем процессы 

    index_of_uncovered_RC = [i for i, (a, b) in enumerate(zip([sum(col) for col in zip(*q_without_broken_prod_node_all)], np.array(data['mu']))) if a != b][0]
    print(index_of_uncovered_RC)
    uncovered_demands = np.array([row[index_of_uncovered_RC] for row in np.array(data['d_star_jk'])])
    covered_demands = np.array([row[1 - index_of_uncovered_RC] for row in np.array(data['d_star_jk'])])
    demand_nodes_related_to_uncovered_RC = [i for i, row in enumerate(np.array(data['d_star_jk'])) if row[index_of_uncovered_RC] != 0]
    available_in_uncovered_RC = [sum(col) for col in zip(*q_without_broken_prod_node_all)]
    transp_matrix_1_for_first_PC = [row[0] for row in q_without_broken_prod_node_all]
    if sum(transp_matrix_1_for_first_PC) == 0:
        transp_matrix_2_for_first_PC = np.zeros(n)
    else:
        transp_matrix_2_for_first_PC = [row[0] for row in data['d_star_jk']]
    Ct_ij_1 = [row[0] for row in data['Ct_ij']]
    Cr_jk_1 = [row[0] for row in data['Cr_jk_transposed']]
    index_of_working_distr_node_1 = 0 
    transp_matrix_1_for_second_PC = [row[1] for row in q_without_broken_prod_node_all]
    if sum(transp_matrix_1_for_second_PC) == 0:
        transp_matrix_2_for_second_PC = np.zeros(n)
    else:
        transp_matrix_2_for_second_PC = [row[1] for row in data['d_star_jk']]
    Ct_ij_2 = [row[1] for row in data['Ct_ij']]
    Cr_jk_2 = [row[1] for row in data['Cr_jk_transposed']]
    index_of_working_distr_node_2 = 1
    demand_nodes_related_to_PC_1 = [i for i, row in enumerate(data['d_star_jk']) if row[0] != 0 and row[1] == 0]
    demand_nodes_related_to_PC_2 = [i for i, row in enumerate(data['d_star_jk']) if row[1] != 0 and row[0] == 0]
    demand_nodes_related_to_both = [i for i, row in enumerate(data['d_star_jk']) if row[1] != 0 and row[0] != 0]

    # сценарий 1.1: поровну всем 
   
    a = scenario_1_new(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC, n, all_gen_data)
    if a is None:
        total_profit_2_1 = None
        print('не можем использовать сценарий 2.1.')
    else:
        if sum(transp_matrix_1_for_first_PC) == 0:
            cost_equall = sum(a * data['Cr_jk'][1])
        elif sum(transp_matrix_1_for_second_PC) == 0:
            cost_equall = sum(a * data['Cr_jk'][0])
        else:
            cost_equall, matrix_between_distr_and_demand_1 = optimal_transportation(a, q_without_broken_prod_node, data['Cr_jk_transposed'])
        total_cost_equall = np.round (sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                             sum(np.array(data['Cd_j']) * q_without_broken_prod_node) + 
                             sum(a * np.array(data['Cs_k'])) + 
                             sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(q_without_broken_prod_node_all)))) + cost_equall, decimals = 2)

        total_value_equall = np.round(sum(a * np.array(data['s_k'])), decimals = 2)
        total_profit_1_1 = np.round(total_value_equall - total_cost_equall, decimals = 2)
        print('сценарий 1.1:', total_profit_1_1) 
    
    # сценарий 1.2: весовые коэффициенты
    int_prop = scenario_2_new(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC, n, all_gen_data)
    if sum(transp_matrix_1_for_first_PC) == 0:
            cost_weight = sum(int_prop * data['Cr_jk'][1])
    elif sum(transp_matrix_1_for_second_PC) == 0:
        cost_weight = sum(int_prop * data['Cr_jk'][0])
    else:
        cost_weight, matrix_between_distr_and_demand_1 = optimal_transportation(int_prop, q_without_broken_prod_node, data['Cr_jk_transposed'])
    total_cost_weight = np.round (sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                             sum(np.array(data['Cd_j']) * q_without_broken_prod_node) + 
                             sum(int_prop * np.array(data['Cs_k'])) + 
                             sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(q_without_broken_prod_node_all)))) + cost_weight, decimals = 2)
    total_value_weight = np.round(sum(int_prop * np.array(data['s_k'])), decimals = 2)
    total_profit_1_2 = np.round(total_value_weight - total_cost_weight, decimals = 2)
    print('сценарий 1.2:', total_profit_1_2)
    
    filling = scenario_3_new(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC, n, all_gen_data)
    if sum(transp_matrix_1_for_first_PC) == 0:
            cost_h = sum(filling * data['Cr_jk'][1])  
    elif sum(transp_matrix_1_for_second_PC) == 0:
        cost_h = sum(filling * data['Cr_jk'][0])
    else:
        cost_h, matrix_between_distr_and_demand_1 = optimal_transportation(filling, q_without_broken_prod_node, data['Cr_jk_transposed'])
    total_cost_h = np.round (sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                             sum(np.array(data['Cd_j']) * q_without_broken_prod_node) + 
                             sum(filling * np.array(data['Cs_k'])) + 
                             sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(q_without_broken_prod_node_all)))) + cost_h, decimals = 2)

    total_value_h = np.round(sum(filling * np.array(data['s_k'])), decimals = 2)
    total_profit_1_3 = np.round(total_value_h - total_cost_h, decimals = 2)
    print('сценарий 1.3:', total_profit_1_3)
    
    # повышаем мощности производств
    print('2. повышение мощностей производств')
    u = np.zeros(m)
    Ct_ij_for_uncovered_RC  = [row[index_of_uncovered_RC] for row in data['Ct_ij']]
    
    for i in range(m):
        u[i] = data['Cp_i'][i] + data['Cp_i_add'][i] + Ct_ij_for_uncovered_RC[i]
    nonzero_indices_add = [i for i in range(m) if i != 0]
    sorted_indices_add = sorted(nonzero_indices_add, key=lambda i: u[i])
    deficit = data['D'] - sum(available_in_uncovered_RC)
    increased_nu = np.zeros(m)
    for i in sorted_indices_add:
        increased_nu[i] =  data['nu_add'][i]
    distribution_of_increased_production = np.zeros((m, 2))
    remainder_add = np.array(data['mu_add']) 
   
    for i in range(m):
        # определяем предпочтительный центр для направления доп продукции
        if data['Ct_ij'][i][0] + data['Cd_j'][0] > data['Ct_ij'][i][1] + data['Cd_j'][1]:
            preferred_j = 1
            alternative_j = 0
        else:
            preferred_j = 0
            alternative_j = 1
        
        # распределяем в предпочтительный 
        if remainder_add[preferred_j] > 0:
            if remainder_add[preferred_j] >= increased_nu[i]:
                distribution_of_increased_production[i][preferred_j] = increased_nu[i]
                remainder_add[preferred_j] -= increased_nu[i]
            else:
                
                distributed = remainder_add[preferred_j]
                distribution_of_increased_production[i][preferred_j] = distributed
                remainder_add[preferred_j] = 0
                
                # остаток пытаемся распределить в другой рц
                remaining = increased_nu[i] - distributed
                if remainder_add[alternative_j] >= remaining:
                    distribution_of_increased_production[i][alternative_j] += remaining
                    remainder_add[alternative_j] -= remaining
                else:
                    distribution_of_increased_production[i][alternative_j] += remainder_add[alternative_j]
                    remainder_add[alternative_j] = 0
        else:
            # если предпочтительный центр уже заполнен полностью
            if remainder_add[alternative_j] >= increased_nu[i]:
                distribution_of_increased_production[i][alternative_j] = increased_nu[i]
                remainder_add[alternative_j] -= increased_nu[i]
            else:
                distribution_of_increased_production[i][alternative_j] = remainder_add[alternative_j]
                remainder_add[alternative_j] = 0
    
    distribution_of_increased_production = distribution_of_increased_production + q_without_broken_prod_node_all
    distribution_of_increased_production_not_all = [sum(x) for x in zip(*distribution_of_increased_production)]
    q_without_broken_prod_node_all_add = copy.deepcopy(data['nu_add'])
    q_without_broken_prod_node_all_add[index_of_broken_prod_node] = 0 
    
    q = [sum(x) for x in zip(*distribution_of_increased_production)]
    add_cost_in_PC = np.zeros(2)
    for j in range(2):
        if q[j] > data['mu'][j]:
            add_cost_in_PC[j] = (q[j] - data['mu'][j]) * data['Cd_j_add'][j]
   

    a = scenario_1_new(np.sum(distribution_of_increased_production), n, all_gen_data)
    if a is None:
        print('не можем использовать сценарий 2.1.')
        total_profit_2_1 = None
    else:        
        if q[0] == 0:
            cost_equall = sum(a * data['Cr_jk'][1])
        elif q[1] == 0:
            cost_equall = sum(a * data['Cr_jk'][0])
        else:
            cost_equall, matrix_between_distr_and_demand_1 = optimal_transportation( a, q, data['Cr_jk_transposed'])
        total_cost_equall = np.round(sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                             sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all_add), np.array(data['Cp_i_add']))) +
                             sum(np.array(data['Cd_j']) * distribution_of_increased_production_not_all) + np.sum(add_cost_in_PC) +
                             sum(a * np.array(data['Cs_k'])) + 
                             sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(distribution_of_increased_production)))) + cost_equall, decimals = 2)
        total_value_equall = np.round(sum(a * np.array(data['s_k'])), decimals = 2)
        total_profit_2_1 = np.round(total_value_equall - total_cost_equall, decimals = 2)
        print('сценарий 2.1:', total_profit_2_1) 
    

    # сценарий 1.2: весовые коэффициенты
    int_prop = scenario_2_new(np.sum(distribution_of_increased_production), n, all_gen_data)
    if q[0] == 0:
        cost_weight = sum(int_prop * data['Cr_jk'][1])
    elif q[1] == 0:
        cost_weight = sum(int_prop * data['Cr_jk'][0])
    else:
        cost_weight, matrix_between_distr_and_demand_1 = optimal_transportation(int_prop, q, data['Cr_jk_transposed'])
    total_cost_weight = np.round(sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                             sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all_add), np.array(data['Cp_i_add']))) +
                             sum(np.array(data['Cd_j']) * distribution_of_increased_production_not_all) + np.sum(add_cost_in_PC) +
                             sum(int_prop * np.array(data['Cs_k'])) + 
                             sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(distribution_of_increased_production)))) + cost_weight, decimals = 2)
    total_value_weight = np.round(sum(int_prop * np.array(data['s_k'])), decimals = 2)
    total_profit_2_2 = np.round(total_value_weight - total_cost_weight, decimals = 2)
    print('сценарий 2.2:', total_profit_2_2)
   
    filling = scenario_3_new(np.sum(distribution_of_increased_production), n, all_gen_data)
    if q[0] == 0:
        cost_h = sum(filling * data['Cr_jk'][1])
    elif q[1] == 0:
        cost_h = sum(filling * data['Cr_jk'][0])
    else:
        cost_h, matrix_between_distr_and_demand_1 = optimal_transportation(filling, q, data['Cr_jk_transposed'])
    total_cost_h = np.round(sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                             sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all_add), np.array(data['Cp_i_add']))) +
                             sum(np.array(data['Cd_j']) * distribution_of_increased_production_not_all) + np.sum(add_cost_in_PC) +
                             sum(filling * np.array(data['Cs_k'])) + 
                             sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(distribution_of_increased_production)))) + cost_h, decimals = 2)
    
    total_value_h = np.round(sum(filling * np.array(data['s_k'])), decimals = 2)
    total_profit_2_3 = np.round(total_value_h - total_cost_h, decimals = 2)
    print('сценарий 2.3:', total_profit_2_3)
    rez = [
    total_profit_1_1 if 'total_profit_1_1' in locals() else None,
    total_profit_1_2 if 'total_profit_1_2' in locals() else None,
    total_profit_1_3 if 'total_profit_1_3' in locals() else None,
    total_profit_2_1 if 'total_profit_2_1' in locals() else None,
    total_profit_2_2 if 'total_profit_2_2' in locals() else None,
    total_profit_2_3 if 'total_profit_2_3' in locals() else None
]
    return rez

In [ ]:
# генерируем матожидание t в каждом производстенном и распределительном узле
LOWER_BOUND_k_disruption = 0.5
UPPER_BOUND_k_disruption = 1.9

LOWER_BOUND_lambda_disruption = 1
UPPER_BOUND_lambda_disruption = 100

LOWER_BOUND_number_of_samples = 20
UPPER_BOUND_number_of_samples = 60

LOWER_BOUND_k_working = 0.9
UPPER_BOUND_k_working = 1.9

LOWER_BOUND_lambda_working = 10
UPPER_BOUND_lambda_working = 100

def generate_weibull_params(size, LOWER_BOUND_k, UPPER_BOUND_k, 
                          LOWER_BOUND_lambda, UPPER_BOUND_lambda,
                          LOWER_BOUND_number_of_samples, UPPER_BOUND_number_of_samples):
    
    k = np.random.uniform(LOWER_BOUND_k, UPPER_BOUND_k, size=size)
    lambda_ = np.random.randint(LOWER_BOUND_lambda, UPPER_BOUND_lambda, size=size)
    number_of_samples = np.random.randint(LOWER_BOUND_number_of_samples, 
                                        UPPER_BOUND_number_of_samples, 
                                        size=size)
    
    
    t = [np.round(weibull_min.rvs(k[i], scale=lambda_[i], size=number_of_samples[i])).astype(int)
         for i in range(size)]
    
    E = np.round(lambda_ * gamma(1 + 1 / k)).astype(int)
    return k, lambda_, number_of_samples, t, E

# Генерация времени сбоев
k_prod_disruption, lambda_prod_disruption, number_of_samples_prod_disruption, t_prod_disruption, E_prod_hour_disruption = generate_weibull_params(
    m, LOWER_BOUND_k_disruption, UPPER_BOUND_k_disruption, LOWER_BOUND_lambda_disruption, UPPER_BOUND_lambda_disruption,
    LOWER_BOUND_number_of_samples, UPPER_BOUND_number_of_samples)
print("E[T] - время в часах сбоя в производственных узлах = ", E_prod_hour_disruption)

k_distr_disruption, lambda_distr_disruption, number_of_samples_distr_disruption, t_distr_disruption, E_distr_hour_disruption = generate_weibull_params(
    2, LOWER_BOUND_k_disruption, UPPER_BOUND_k_disruption, LOWER_BOUND_lambda_disruption, UPPER_BOUND_lambda_disruption,
    LOWER_BOUND_number_of_samples, UPPER_BOUND_number_of_samples)
print("E[T] - время в часах сбоя в распределеительных узлах = ", E_distr_hour_disruption)

E_prod_disruption = np.ceil(E_prod_hour_disruption / 12)
E_distr_disruption = np.ceil(E_distr_hour_disruption / 12)
print(E_prod_disruption, E_distr_disruption)

# Генерация времени работы
k_prod_working, lambda_prod_working, number_of_samples_prod_working, t_prod_working, E_prod_hour_working = generate_weibull_params(
    m, LOWER_BOUND_k_working, UPPER_BOUND_k_working, LOWER_BOUND_lambda_working, UPPER_BOUND_lambda_working,
    LOWER_BOUND_number_of_samples, UPPER_BOUND_number_of_samples)
print("E[T] - дни работы в производственных узлах = ", E_prod_hour_working)

k_distr_working, lambda_distr_working, number_of_samples_distr_working, t_distr_working, E_distr_hour_working = generate_weibull_params(
    2, LOWER_BOUND_k_working, UPPER_BOUND_k_working, LOWER_BOUND_lambda_working, UPPER_BOUND_lambda_working,
    LOWER_BOUND_number_of_samples, UPPER_BOUND_number_of_samples)
print("E[T] - дни работы в распределеительных узлах = ", E_distr_hour_working)


In [13]:
def stocks(m, n, all_gen_data, E_distr, E_prod, E_distr_hour_working, E_prod_hour_working):
    losses_from_deficit_demand_node = all_gen_data['profit_per_unit']
    losses_from_deficit_distr_node = np.zeros(2)
    losses_from_deficit_prod_node = np.zeros(m)
    sum_2 = np.zeros(n)
    for j in range(2):
        for k in range(n):
            sum_2[k] = losses_from_deficit_demand_node[k] * all_gen_data['d_star_jk'][k][j]
        losses_from_deficit_distr_node[j] = np.sum(sum_2)/all_gen_data['mu'][j]
    sum_1 = np.zeros(m)
    for i in range(m):
        for j in range(2):
            sum_1[j] = losses_from_deficit_distr_node[j] * all_gen_data['q_ij'][i][j]
        losses_from_deficit_prod_node[i] =  np.sum(sum_1)/all_gen_data['nu'][j]
    print(losses_from_deficit_prod_node)
    S_prod = np.zeros(m)
    S_distr = np.zeros(2)
    S_demand =  np.zeros(n)
    # for i in range(m):
    #     if (all_gen_data['nu'][i] * E_prod[i]) * (1 - (all_gen_data['Hp_i'][i] / losses_from_deficit_prod_node[i])) > 0:
    #         S_prod[i] = min(np.floor((all_gen_data['nu'][i] * E_prod[i]) * (1 - (all_gen_data['Hp_i'][i] / losses_from_deficit_prod_node[i]))), all_gen_data['alpha'][i])

    # for j in range(2):
    #     if ((all_gen_data['mu'][j] * E_distr[j]) * (1 - (all_gen_data['Hd_j'][j] /losses_from_deficit_distr_node[j]))) > 0:
    #         S_distr[j] = min(np.floor(((all_gen_data['mu'][j] * E_distr[j]) * (1 - (all_gen_data['Hd_j'][j] /losses_from_deficit_distr_node[j])))), all_gen_data['beta'][j])
    for i in range(m):
        if (all_gen_data['nu'][i] * E_prod[i]) * (1 - ((all_gen_data['Hp_i'][i] * E_prod_hour_working[i]) / (E_prod[i] * losses_from_deficit_prod_node[i]))) > 0:
            S_prod[i] = min(np.floor((all_gen_data['nu'][i] * E_prod[i]) * (1 - ((all_gen_data['Hp_i'][i] * E_prod_hour_working[i]) / (E_prod[i] * losses_from_deficit_prod_node[i])))), all_gen_data['alpha'][i])

    for j in range(2):
        if ((all_gen_data['mu'][j] * E_distr[j]) * (1 - ((all_gen_data['Hd_j'][j] * E_distr_hour_working[j]) /(E_distr[j] *losses_from_deficit_distr_node[j])))) > 0:
            S_distr[j] = min(np.floor((all_gen_data['mu'][j] * E_distr[j]) * (1 - ((all_gen_data['Hd_j'][j] * E_distr_hour_working[j]) /(E_distr[j] *losses_from_deficit_distr_node[j])))), all_gen_data['beta'][j])
    print(losses_from_deficit_distr_node, losses_from_deficit_prod_node )
    return S_prod, S_distr


In [ ]:
safety_stocks_prod, safety_stocks_distr = stocks(m, n, all_gen_data, E_distr_disruption, E_prod_disruption, E_distr_hour_working, E_prod_hour_working)
print(safety_stocks_prod, safety_stocks_distr)

def generate_failure(production_nodes_count, distribution_nodes_count, all_gen_data ):
    node_type = np.random.choice(['производственном', 'распределительном'])
    if node_type == 'производственном':
        node_index = np.random.randint(0, production_nodes_count)
        # d_3 = production_node_disruption(m, n, node_index, all_gen_data)
    else:
        node_index = np.random.randint(0, distribution_nodes_count)
        # d_2 = distribution_node_disruption(m, n, node_index, all_gen_data)
    return node_type, node_index

failure_type, failed_node = generate_failure(m, 2, all_gen_data)
print('сбой произошел в', failed_node, failure_type, 'центре.')

# failure_type, failed_node = 'распределительном', 1
# failure_type, failed_node = 'производственном', 0

In [15]:
def send_stock(data, index_of_broken_prod_node, safety_stocks_prod):
    # это от пц до рц
    q_without_broken_prod_node_all = copy.deepcopy(data['q_ij'])
    q_without_broken_prod_node_all[index_of_broken_prod_node] = [0, 0]
    q_without_broken_prod_node = [sum(x) for x in zip(*q_without_broken_prod_node_all)]
    
    # index_of_uncovered_RC = [i for i, (a, b) in enumerate(zip([sum(col) for col in zip(*q_without_broken_prod_node_all)], np.array(data['mu']))) if a != b][0]
    transp_matrix_1_for_first_PC = [row[0] for row in q_without_broken_prod_node_all]
    transp_matrix_1_for_second_PC = [row[1] for row in q_without_broken_prod_node_all]

    distribution_of_production = np.zeros((m, 2))
    remainder_add = np.array(data['mu']) - q_without_broken_prod_node
    i = index_of_broken_prod_node

    if data['Ct_ij'][i][0] + data['Cd_j'][0]> data['Ct_ij'][i][1] + data['Cd_j'][1] :
        preferred_j = 1
        alternative_j = 0
    else:
        preferred_j = 0
        alternative_j = 1
    
    if remainder_add[preferred_j] > 0:
        if remainder_add[preferred_j] >= safety_stocks_prod[i]:
            distribution_of_production[i][preferred_j] = safety_stocks_prod[i]
            remainder_add[preferred_j] -= safety_stocks_prod[i]
        else:
            distributed = remainder_add[preferred_j]
            distribution_of_production[i][preferred_j] = distributed
            remainder_add[preferred_j] = 0
            
            remaining = safety_stocks_prod[i] - distributed
            if remainder_add[alternative_j] >= remaining:
                distribution_of_production[i][alternative_j] += remaining
                remainder_add[alternative_j] -= remaining
            else:
                distribution_of_production[i][alternative_j] += remainder_add[alternative_j]
                remainder_add[alternative_j] = 0
    else:
        if remainder_add[alternative_j] >= safety_stocks_prod[i]:
            distribution_of_production[i][alternative_j] = safety_stocks_prod[i]
            remainder_add[alternative_j] -= safety_stocks_prod[i]
        else:
            distribution_of_production[i][alternative_j] = remainder_add[alternative_j]
            remainder_add[alternative_j] = 0
    return distribution_of_production + q_without_broken_prod_node_all


In [16]:
def distribution_stocks_prod(distribution_production, data, q_without_broken_prod_node_all, q):
    # Сценарии 2.1, 2.2, 2.3
    rez = np.zeros(3)
    scenario_index = 0
    for scenario_name, scenario_func in [('2.1', scenario_1_new), ('2.2', scenario_2_new), ('2.3', scenario_3_new)]:
        distribution_vector = scenario_func(np.sum(distribution_production), n, all_gen_data)
        if distribution_vector is None:
            # print (f'не можем использовать сценарий {scenario_name}.')
            total_profit = None
            continue

            
        if q[0] == 0:
            cost_transport = sum(distribution_vector * data['Cr_jk'][1])
        elif q[1] == 0:
            cost_transport = sum(distribution_vector * data['Cr_jk'][0])
        else:
            cost_transport, _ = optimal_transportation(distribution_vector, q, data['Cr_jk_transposed'])
        
        add_cost_in_PC = np.zeros(2)
        for j in range(2):
            if q[j] > data['mu'][j]:
                add_cost_in_PC[j] = (q[j] - data['mu'][j]) * data['Cd_j_add'][j]

        total_cost = np.round(
                sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) +
                sum(np.array(data['Cd_j']) * [sum(x) for x in zip(*distribution_production)]) + np.sum(add_cost_in_PC) +
                sum(distribution_vector * np.array(data['Cs_k'])) + 
                sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(distribution_production)))) + 
                cost_transport,
                decimals=2)        
        total_value = np.round(sum(distribution_vector * np.array(data['s_k'])), decimals=2)
        total_profit = np.round(total_value - total_cost, decimals=2)
       

        print(f'сценарий {scenario_name}:', total_profit)
        if total_profit is None:
            print(f"Предупреждение: total_profit=None для сценария {scenario_name}")
            rez[scenario_index] = 0.0  # Или другое значение по умолчанию
        elif isinstance(total_profit, (np.ndarray, list)):
            rez[scenario_index] = float(np.sum(total_profit))  # Суммируем если это массив
        else:
            rez[scenario_index] = float(total_profit)  # Явное преобразование к float
        
        scenario_index += 1
        # print(f'Сценарий {scenario_name}:', rez[scenario_index-1])
    return(rez)
        # return (f'сценарий {scenario_name}:', total_profit)

In [17]:
def distribution_stocks(distribution_production, data, q_without_broken_prod_node_all, q):
    # Сценарии 2.1, 2.2, 2.3
    rez = np.zeros(3)
    scenario_index = 0
    for scenario_name, scenario_func in [('2.1', scenario_1_new), ('2.2', scenario_2_new), ('2.3', scenario_3_new)]:
        distribution_vector = scenario_func(np.sum(distribution_production), n, all_gen_data)
        if distribution_vector is None:
            # print (f'не можем использовать сценарий {scenario_name}.')
            total_profit = None
            continue

            
        if q[0] == 0:
            cost_transport = sum(distribution_vector * data['Cr_jk'][1])
        elif q[1] == 0:
            cost_transport = sum(distribution_vector * data['Cr_jk'][0])
        else:
            cost_transport, _ = optimal_transportation(distribution_vector, q, data['Cr_jk_transposed'])
        
        add_cost_in_PC = np.zeros(2)
        for j in range(2):
            if q[j] > data['mu'][j]:
                add_cost_in_PC[j] = (q[j] - data['mu'][j]) * data['Cd_j_add'][j]

       
        total_cost = np.round(
                sum((x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) +
                sum(np.array(data['Cd_j']) * [sum(x) for x in zip(*distribution_production)]) + np.sum(add_cost_in_PC) +
                sum(distribution_vector * np.array(data['Cs_k'])) + 
                sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(distribution_production)))) + 
                cost_transport,
                decimals=2)        
        total_value = np.round(sum(distribution_vector * np.array(data['s_k'])), decimals=2)
        total_profit = np.round(total_value - total_cost, decimals=2)
       

        print(f'сценарий {scenario_name}:', total_profit)
        if total_profit is None:
            print(f"Предупреждение: total_profit=None для сценария {scenario_name}")
            rez[scenario_index] = 0.0  # Или другое значение по умолчанию
        elif isinstance(total_profit, (np.ndarray, list)):
            rez[scenario_index] = float(np.sum(total_profit))  # Суммируем если это массив
        else:
            rez[scenario_index] = float(total_profit)  # Явное преобразование к float
        
        scenario_index += 1
        # print(f'Сценарий {scenario_name}:', rez[scenario_index-1])
    return(rez)
        # return (f'сценарий {scenario_name}:', total_profit)

In [18]:
T_disruption = 3

In [ ]:
q_without_broken_prod_node_all = copy.deepcopy(all_gen_data['q_ij'])
q_without_broken_prod_node_all[failed_node] = [0, 0]
q_without_broken_prod_node = [sum(x) for x in zip(*q_without_broken_prod_node_all)]
# print(q_without_broken_prod_node_all, q_without_broken_prod_node)
if sum(all_gen_data['alpha'] + all_gen_data['beta']) > 0:
    print('модель может использовать запасы.')
    if failure_type == 'производственном':
        if safety_stocks_prod[failed_node] != 0:
            enough_stock_for_days = int((safety_stocks_prod[failed_node] / all_gen_data['nu'][failed_node]))

            # стоимость хранения запасов во время сбоев
            cost_of_stocks =  np.sum(safety_stocks_distr * all_gen_data['Hd_j']) + np.sum(safety_stocks_prod * all_gen_data['Hp_i']) 

            cost_of_stocks_without_one_stock =  np.sum(safety_stocks_distr * all_gen_data['Hd_j']) + np.sum(safety_stocks_prod * all_gen_data['Hp_i']) - np.sum(safety_stocks_prod[failed_node] * all_gen_data['Hp_i'][failed_node])
            cost = 0
            for t in range (enough_stock_for_days):
                cost += cost_of_stocks_without_one_stock + (safety_stocks_prod[failed_node] - (t+1) * all_gen_data['nu'][failed_node]) * all_gen_data['Hp_i'][failed_node]
            cost_of_stocks_without_one_stock = cost + cost_of_stocks_without_one_stock * (T_disruption - enough_stock_for_days)
                
            if enough_stock_for_days >= T_disruption:
                life_cycle_cost = np.round((E_prod_hour_working[failed_node] * (all_gen_data['total_profit'] - cost_of_stocks) - 
                                           cost_of_stocks_without_one_stock -
                                           np.sum(all_gen_data['Cp_i_add'][failed_node] * all_gen_data['nu'][failed_node] * T_disruption)), decimals=2)

            else: 
                profit_on_stocks = all_gen_data['total_profit'] * enough_stock_for_days
                rest_of_stock = safety_stocks_prod - enough_stock_for_days * np.array(all_gen_data['nu'])    
                distribute_production = send_stock(all_gen_data, failed_node, rest_of_stock)
                q = [sum(x) for x in zip(*distribute_production)]
                profit_part_of_stocks = distribution_stocks_prod(distribute_production, all_gen_data, q_without_broken_prod_node_all, q)
                profit_without_stocks = production_node_disruption(m, n, failed_node, all_gen_data)
                profit_without_stocks_max = max(filter(lambda x: x is not None, profit_without_stocks))
                profit_part_of_stocks_max = max(filter(lambda x: x is not None, profit_part_of_stocks))
                
                print('стоимость по частям',profit_on_stocks, profit_part_of_stocks_max, profit_without_stocks_max)
                print('за хранение', E_prod_hour_working[failed_node]*cost_of_stocks, cost_of_stocks_without_one_stock)
                life_cycle_cost = np.round(E_prod_hour_working[failed_node] * (all_gen_data['total_profit'] - cost_of_stocks) +
                                           profit_on_stocks * enough_stock_for_days + profit_part_of_stocks_max + 
                                           profit_without_stocks_max * (T_disruption - enough_stock_for_days - 1) - 
                                           cost_of_stocks_without_one_stock - np.sum(all_gen_data['Cp_i_add'][failed_node] * safety_stocks_prod[failed_node]), decimals=2)
            print('СТОИМОСТЬ ЦИКЛА с запасами', life_cycle_cost)
            profit_without_stocks = production_node_disruption(m, n, failed_node, all_gen_data)
            profit_without_stocks_max = max(filter(lambda x: x is not None, profit_without_stocks))
    
            cycle_without_stocks = np.round(E_prod_hour_working[failed_node] * all_gen_data['total_profit'] + profit_without_stocks_max * T_disruption, decimals=2)
            print('СТОИМОСТЬ ЦИКЛА без запаса', cycle_without_stocks)
                
    
    if failure_type == 'распределительном':
        if safety_stocks_distr[failed_node] != 0:

            transp_matrix_with_broken = [row[failed_node] for row in all_gen_data['q_ij']]

            enough_stock_for_days = int((safety_stocks_distr[failed_node] / all_gen_data['mu'][failed_node]))
            

            E_p = np.zeros(m) 
            cost_add_for_new_stock = np.zeros(m) 
            for i in range(m):
                cost_add_for_new_stock[i] = all_gen_data['Cp_i_add'][i] + all_gen_data['Ct_ij'][i][failed_node]
            sorted_indices = sorted(range(m), key=lambda i: (cost_add_for_new_stock[i]))

            cost_of_stocks =  np.sum(safety_stocks_distr * all_gen_data['Hd_j']) + np.sum(safety_stocks_prod * all_gen_data['Hp_i']) 
    
            cost_of_stocks_without_one_stock =  np.sum(safety_stocks_distr * all_gen_data['Hd_j']) + np.sum(safety_stocks_prod * all_gen_data['Hp_i']) - np.sum(safety_stocks_distr[failed_node] * all_gen_data['Hd_j'][failed_node])
            cost = 0
            for t in range (enough_stock_for_days):
                cost += cost_of_stocks_without_one_stock + (safety_stocks_distr[failed_node] - (t + 1) * all_gen_data['mu'][failed_node]) * all_gen_data['Hd_j'][failed_node]
            cost_of_stocks_without_one_stock = cost + cost_of_stocks_without_one_stock * (T_disruption - enough_stock_for_days)
                           
            index_of_working_distr_node = 1 - failed_node
            transp_matrix_without_broken_1 = [row[index_of_working_distr_node] for row in all_gen_data['q_ij']]
            transp_matrix_with_broken_1 = [row[failed_node] for row in all_gen_data['q_ij']]  


            if enough_stock_for_days >= T_disruption:
                compensation_stocks = np.zeros(m)
                for i in sorted_indices:
                    rem = T_disruption * all_gen_data['mu'][failed_node]
                    if all_gen_data['nu_add'][i] < rem and rem != 0:
                        compensation_stocks[i] = all_gen_data['nu_add'][i] 
                        rem -= all_gen_data['nu_add'][i]
                    else:
                        compensation_stocks[i] = rem
                        rem = 0
                profit_on_stocks = all_gen_data['total_profit'] * enough_stock_for_days
                life_cycle_cost = max(np.round((E_distr_hour_working[failed_node] * (all_gen_data['total_profit'] - cost_of_stocks) + profit_on_stocks * T_disruption
                                            - compensation_stocks * all_gen_data['Cp_i_add']), decimals=2) )           
            else:
                profit_on_stocks = all_gen_data['total_profit'] * enough_stock_for_days
                rest_of_stock = safety_stocks_distr - enough_stock_for_days * np.array(all_gen_data['mu'])    
                if failed_node == 0:
                    distribute_production = [[int(rest_of_stock[0])], [int(all_gen_data['mu'][1])]]
                else:
                    distribute_production =  [[int(all_gen_data['mu'][0])], [int(rest_of_stock[1])]]
                q = [x[0] for x in distribute_production]
                # print(q)
                profit_part_of_stocks = distribution_stocks(distribute_production, all_gen_data, transp_matrix_without_broken_1, q)
                profit_without_stocks = distribution_node_disruption(m, n, failed_node, all_gen_data)
                profit_without_stocks_max = max(filter(lambda x: x is not None, profit_without_stocks))
                profit_part_of_stocks_max = max(filter(lambda x: x is not None, profit_part_of_stocks))
                print('стоимость по частям',profit_on_stocks, profit_part_of_stocks_max, profit_without_stocks_max)

    
                # print(profit_on_stocks, profit_part_of_stocks_max, profit_without_stocks_max)
                compensation_stocks = np.zeros(m)
                for i in sorted_indices:
                    rem = safety_stocks_distr[failed_node]
                    if all_gen_data['nu_add'][i] < rem and rem != 0:
                        compensation_stocks[i] = all_gen_data['nu_add'][i] 
                        rem -= all_gen_data['nu_add'][i]
                    else:
                        compensation_stocks[i] = rem
                        rem = 0
                
                
                life_cycle_cost = np.round(E_distr_hour_working[failed_node] * (all_gen_data['total_profit'] - cost_of_stocks) + 
                                           profit_on_stocks * enough_stock_for_days + profit_part_of_stocks_max + 
                                           profit_without_stocks_max * (T_disruption - enough_stock_for_days - 1) - 
                                           cost_of_stocks_without_one_stock - np.sum(all_gen_data['Cp_i_add'] * compensation_stocks) - 
                                           E_prod_disruption[failed_node] * sum(x * y for x, y in zip(np.array(transp_matrix_with_broken_1), np.array(all_gen_data['C_stop_i'])))  , decimals=2)
                print('стоимость по частям', profit_on_stocks, profit_part_of_stocks_max, profit_without_stocks_max, 'd', all_gen_data['total_profit'] - cost_of_stocks)



            print('СТОИМОСТЬ ЦИКЛА с запасами', life_cycle_cost)
            profit_without_stocks = distribution_node_disruption(m, n, failed_node, all_gen_data)
            profit_without_stocks_max = max(filter(lambda x: x is not None, profit_without_stocks))
            cycle_without_stocks = np.round(E_distr_hour_working[failed_node] * all_gen_data['total_profit'] + profit_without_stocks_max * T_disruption, decimals=2)
            print('СТОИМОСТЬ ЦИКЛА без запаса', cycle_without_stocks)
